In [4]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.prompt_values import ChatPromptValue

from langchain_core.messages.base import BaseMessage

template: ChatPromptTemplate = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}",
        ),
        MessagesPlaceholder("history", n_messages=2),
        ("human", "{text}"),
    ]
)

value: ChatPromptValue = template.format_prompt(
    input_language="English",
    output_language="French",
    history=[("human", "what's 5 + 2"), ("ai", "5 + 2 is 7")],
    text="I love programming.",
)

messages: list[BaseMessage] = value.to_messages()
for m in messages:
    print(type(m))
    print(m.get_lc_namespace())
    print(m)


<class 'langchain_core.messages.system.SystemMessage'>
['langchain', 'schema', 'messages']
content='You are a helpful assistant that translates English to French' additional_kwargs={} response_metadata={}
<class 'langchain_core.messages.human.HumanMessage'>
['langchain', 'schema', 'messages']
content="what's 5 + 2" additional_kwargs={} response_metadata={}
<class 'langchain_core.messages.ai.AIMessage'>
['langchain', 'schema', 'messages']
content='5 + 2 is 7' additional_kwargs={} response_metadata={}
<class 'langchain_core.messages.human.HumanMessage'>
['langchain', 'schema', 'messages']
content='I love programming.' additional_kwargs={} response_metadata={}


In [5]:
import getpass

password = getpass.getpass("hello")
print(password)

ss


In [1]:
import torch
from abc import ABC
from typing import List, Optional

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.models.qwen2.modeling_qwen2 import Qwen2ForCausalLM
from transformers.models.qwen2.tokenization_qwen2_fast import (
    Qwen2TokenizerFast,
)


model_name = "/root/xiatian/models/Qwen2.5-7B-Instruct"
model: Qwen2ForCausalLM = AutoModelForCausalLM.from_pretrained(
    model_name, torch_dtype="auto", device_map="cpu"
)

tokenizer: Qwen2TokenizerFast = AutoTokenizer.from_pretrained(model_name)

device = torch.device("cuda:0")
model = model.to(device)


/root/.cache/pypoetry/virtualenvs/knowpath-0Zuin7zc-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.77it/s]


In [3]:
print(model.device)

cuda:0


In [7]:
model.half()

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm):

In [12]:
prompt = "他一年用掉了三万多把扫帚打扫卫生。"
prompt = "据估计，2019年北京市垃圾处理总吨数将突破1000万吨，处理1吨垃圾的费用约为300元/吨，处理这些垃圾的总费用就高达3亿元。"
messages = [
    {
        "role": "system",
        "content": "你是知路助手，负责解决文字中存在的逻辑错误、数字计算错误以及不符合常识的错误，并给出正确答案。请先进行仔细分析，给出推理步骤，遇到数字，则调用计算器进行计算以便验证。分析完毕后，在最后一行输出修改后的结果，如果句子没有问题，则在最后一行输出：句子无错误",
    },
    {"role": "user", "content": prompt},
]

text: str = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
model_inputs = tokenizer(
    [text], return_tensors="pt", max_length=512, truncation=True
)
model_inputs = model_inputs.to(model.device)
generated_ids = model.generate(**model_inputs, max_new_tokens=512)

print(tokenizer.batch_decode(generated_ids, skip_special_tokens=False)[0])

# 生成的id删除前面和输入相同的id
generated_ids = [
    output_ids[len(input_ids) :]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

generated_texts = tokenizer.batch_decode(
    generated_ids, skip_special_tokens=False
)
print(generated_texts[0])

<|im_start|>system
你是知路助手，负责解决文字中存在的逻辑错误、数字计算错误以及不符合常识的错误，并给出正确答案。请先进行仔细分析，给出推理步骤，遇到数字，则调用计算器进行计算以便验证。分析完毕后，在最后一行输出修改后的结果，如果句子没有问题，则在最后一行输出：句子无错误<|im_end|>
<|im_start|>user
据估计，2019年北京市垃圾处理总吨数将突破1000万吨，处理1吨垃圾的费用约为300元/吨，处理这些垃圾的总费用就高达3亿元。<|im_end|>
<|im_start|>assistant
首先，我们来验证一下处理1000万吨垃圾的总费用是否为3亿元。

1. 1000万吨垃圾乘以每吨300元的处理费用：
\[ 1000 \times 10^6 \text{ 吨} \times 300 \text{ 元/吨} = 300,000,000,000 \text{ 元} \]

2. 将上述金额转换为亿元：
\[ 300,000,000,000 \text{ 元} = 30,000 \text{ 亿元} \]

显然，处理1000万吨垃圾的总费用应为30,000亿元而不是3亿元。因此，原句中的费用数值存在错误。

修改后的结果：据估计，2019年北京市垃圾处理总吨数将突破1000万吨，处理1吨垃圾的费用约为300元/吨，处理这些垃圾的总费用就高达30,000亿元。<|im_end|>
首先，我们来验证一下处理1000万吨垃圾的总费用是否为3亿元。

1. 1000万吨垃圾乘以每吨300元的处理费用：
\[ 1000 \times 10^6 \text{ 吨} \times 300 \text{ 元/吨} = 300,000,000,000 \text{ 元} \]

2. 将上述金额转换为亿元：
\[ 300,000,000,000 \text{ 元} = 30,000 \text{ 亿元} \]

显然，处理1000万吨垃圾的总费用应为30,000亿元而不是3亿元。因此，原句中的费用数值存在错误。

修改后的结果：据估计，2019年北京市垃圾处理总吨数将突破1000万吨，处理1吨垃圾的费用约为300元/吨，处理这些垃圾的总费用就高达30,000亿元。<|im_end|>


这句话中的逻辑有些不合理，因为一把扫帚不可能在一年内被用掉。如果你的意思是他一年用了大量的扫帚，可以这样表述：

- 他一年用掉了三千多把扫帚打扫卫生。
- 他一年更换了三万多把扫帚来打扫卫生。

具体取决于你想要表达的实际情况。
